In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from crewai_tools import SerperDevTool
#os.environ['SERPER_API_KEY']
search_tool=SerperDevTool()
print(type(search_tool))

<class 'crewai_tools.tools.serper_dev_tool.serper_dev_tool.SerperDevTool'>


In [3]:
search_query = "What is the key feature of CrewAI agent collaborations?"
search_results =search_tool.run(query=search_query )

# Print the results
print(f"Search Results for '{search_query}':\n")
search_results

Using Tool: Search the internet with Serper
Search Results for 'What is the key feature of CrewAI agent collaborations?':



{'searchParameters': {'q': 'What is the key feature of CrewAI agent collaborations?',
  'type': 'search',
  'num': 10,
  'engine': 'google'},
 'organic': [{'title': 'Collaboration',
   'link': 'https://docs.crewai.com/en/concepts/collaboration',
   'snippet': "Collaboration in CrewAI enables agents to work together as a team by delegating tasks and asking questions to leverage each other's expertise.",
   'position': 1},
  {'title': 'What is Crew AI: Collaborative Autonomous Agent ...',
   'link': 'https://medium.com/@tahirbalarabe2/what-is-crew-ai-collaborative-autonomous-agent-framework-cbffc7926e1b',
   'snippet': 'Key Features · Role-Based Design Each agent has a defined role, like “coder” or “tester,” with tailored tools and objectives. · Tool Integration',
   'position': 2},
  {'title': 'What is crewAI?',
   'link': 'https://www.ibm.com/think/topics/crew-ai',
   'snippet': "Agents engage with one another through crewAI's inherent delegation and communication mechanisms giving the

In [4]:
search_results.keys()

dict_keys(['searchParameters', 'organic', 'credits'])

In [5]:
from crewai import LLM

perplexity_llm = LLM(
    model="sonar-pro",
    base_url="https://api.perplexity.ai/",
    max_tokens= 500
)

ImportError: Error importing native provider: OPENAI_API_KEY is required

In [ ]:
ollama_llm = LLM(
    model="ollama/llama3.2:latest",
    base_url="http://localhost:11434",
    max_tokens= 251
)

In [ ]:
from crewai import Agent

research_agent = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge information and insights on any subject with comprehensive analysis',
  backstory="""You are an expert researcher with extensive experience in gathering, analyzing, and synthesizing information across multiple domains. 
  Your analytical skills allow you to quickly identify key trends, separate fact from opinion, and produce insightful reports on any topic. 
  You excel at finding reliable sources and extracting valuable information efficiently.""",
  verbose=True,
  allow_delegation=False,
  llm = perplexity_llm,
  tools=[SerperDevTool()]
)

In [ ]:
research_agent

In [ ]:
# Define your agents with roles and goals
# Define the Writer Agent
writer_agent = Agent(
  role='Tech Content Strategist',
  goal='Craft well-structured and engaging content based on research findings',
  backstory="""You are a skilled content strategist known for translating 
  complex topics into clear and compelling narratives. Your writing makes 
  information accessible and engaging for a wide audience.""",
  verbose=True,
  llm = perplexity_llm,
  allow_delegation=True
)

writer_agent

## **Tasks in CrewAI**

In [ ]:
from crewai import Task

research_task = Task(
  description="Analyze the major {topic}, identifying key trends and technologies. Provide a detailed report on their potential impact.",
  agent=research_agent,
  expected_output="A detailed report on {topic}, including trends, emerging technologies, and their impact."
)

In [ ]:
# Create a task for the Writer Agent
writer_task = Task(
  description="Create an engaging blog post based on the research findings about {topic}. Tailor the content for a tech-savvy audience, ensuring clarity and interest.",
  agent=writer_agent,
  expected_output="A 4-paragraph blog post on {topic}, written clearly and engagingly for tech enthusiasts."
)

## CrewAI Workflow

In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=[research_agent, writer_agent],
    tasks=[research_task, writer_task],
    process=Process.sequential,
    verbose=True 
)

In [ ]:
result = crew.kickoff(inputs={"topic": "Latest Generative AI breakthroughs"})

In [ ]:
print(type(result))
dir(result)
result.raw

In [ ]:
result.schema().keys()

In [ ]:
tasks_outputs = result.tasks_output

print("Task Description", tasks_outputs[0].description)
print("*"*75)
print("Output of research task ",tasks_outputs[0])

In [ ]:
print("Writer task description:", tasks_outputs[1].description)
print("*"*75)
print(" \nOutput of writer task:", tasks_outputs[1].raw)

In [ ]:
print("We can get the agent for researcher task: ",tasks_outputs[0].agent)
print("We can get the agent for the writer task: ",tasks_outputs[1].agent)

In [ ]:
token_count = result.token_usage.total_tokens
prompt_tokens = result.token_usage.prompt_tokens
completion_tokens = result.token_usage.completion_tokens

print(f"Total tokens used: {token_count}")
print(f"Prompt tokens: {prompt_tokens} (used for instructions to the model)")
print(f"Completion tokens: {completion_tokens} (generated in response)")